<a href="https://colab.research.google.com/github/ogut77/DataScience/blob/main/ClassificationComp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# for Jupyter-book, we copy data from GitHub, locally, to save Internet traffic,
# you can specify the data/ folder from the root of your cloned
# https://github.com/Yorko/mlcourse.ai repo, to save Internet traffic
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()
from matplotlib import pyplot as plt
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"
df = pd.read_csv(DATA_PATH + "telecom_churn.csv")

df["International plan"] = pd.factorize(df["International plan"])[0]
df["Voice mail plan"] = pd.factorize(df["Voice mail plan"])[0]
df["Churn"] = df["Churn"].astype("int")
states = df["State"]
y = df["Churn"]
X=df.drop(["State", "Churn"], axis=1)


In [ ]:
X

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,1,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,1,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,1,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,0,0,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2
3329,68,415,0,1,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3
3330,28,510,0,1,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2
3331,184,510,1,1,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2


In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=17
)



In [ ]:
from sklearn.metrics import accuracy_score
tree = DecisionTreeClassifier(random_state=17)
tree.fit(X_train, y_train)
# make predictions for test data
tree_pred = tree.predict(X_test)
accuracy_score(y_test, tree_pred) 

0.92

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=17 )
forest.fit(X_train, y_train)
# make predictions for test data
forest_pred = forest.predict(X_test)
accuracy_score(y_test, forest_pred) 

0.95

In [ ]:
# fit model no training data
from xgboost import XGBClassifier
model = XGBClassifier(random_state=17)
model.fit(X_train, y_train)
# make predictions for test data
xgb_pred = model.predict(X_test)
accuracy_score(y_test, xgb_pred)

0.951

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from time import time
t0 = time()
gbm_model = GradientBoostingClassifier(random_state=17)
gbm_model.fit(X_train, y_train)
gbm_time = time() - t0
accuracy_score(y_test, gbm_model.predict(X_test))

0.954

In [ ]:
import lightgbm as lgb
t0 = time()
lgb_model = lgb.LGBMClassifier(random_state=17)
lgb_model.fit(X_train, y_train)
lgb_time = time() - t0
accuracy_score(y_test, lgb_model.predict(X_test))

0.956

In [ ]:
!pip install catboost


In [ ]:
from catboost import CatBoostClassifier
cbc= CatBoostClassifier(random_state=17,verbose=False)

cbc.fit(X_train, y_train)
accuracy_score(y_test, cbc.predict(X_test))


0.955

In [ ]:
combined=np.round((cbc.predict_proba(X_test)[:,1]+lgb_model.predict_proba(X_test)[:,1]+gbm_model.predict_proba(X_test)[:,1])/3)
accuracy_score(y_test, combined)

0.957

In [ ]:
pip install pycaret

     |████████████████████████████████| 301 kB 5.3 MB/s 
     |████████████████████████████████| 1.3 MB 51.9 MB/s 
     |████████████████████████████████| 2.0 MB 38.8 MB/s 
     |████████████████████████████████| 1.7 MB 37.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 114 kB 35.4 MB/s 
     |████████████████████████████████| 261 kB 36.5 MB/s 
     |████████████████████████████████| 167 kB 41.6 MB/s 
     |████████████████████████████████| 56 kB 2.0 MB/s 
     |████████████████████████████████| 6.8 MB 11.7 MB/s 
     |████████████████████████████████| 16.5 MB 130 kB/s 
     |████████████████████████████████| 86 kB 3.7 MB/s 
     |████████████████████████████████| 102 kB 8.9 MB/s 
     |████████████████████████████████| 675 kB 43.1 MB/s 
     |████████████████████████████████| 636 kB 53.1 MB/s 
     |███████████████████

In [ ]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9351,0.9007,0.6922,0.8778,0.7693,0.7324,0.7422,0.399
lightgbm,Light Gradient Boosting Machine,0.9308,0.8828,0.6414,0.8992,0.7411,0.7030,0.7206,0.186
rf,Random Forest Classifier,0.9143,0.9030,0.5049,0.9179,0.6434,0.6004,0.6389,0.620
dt,Decision Tree Classifier,0.9039,0.8160,0.6878,0.6974,0.6907,0.6339,0.6352,0.028
et,Extra Trees Classifier,0.8879,0.8888,0.3798,0.8247,0.5038,0.4527,0.5033,0.656
ada,Ada Boost Classifier,0.8733,0.8623,0.4268,0.6573,0.5128,0.4440,0.4602,0.171
lr,Logistic Regression,0.8512,0.8696,0.2895,0.5526,0.3750,0.3010,0.3231,0.431
lda,Linear Discriminant Analysis,0.8512,0.8750,0.4142,0.5327,0.4634,0.3792,0.3845,0.046
ridge,Ridge Classifier,0.8445,0.0000,0.2735,0.5249,0.3533,0.2752,0.2970,0.016
dummy,Dummy Classifier,0.8432,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.025


In [ ]:
y_train=pd.DataFrame(y_train)
y_train

,Churn
1069,0
1582,0
2444,0
3196,0
267,0
...,...
406,0
1414,0
2191,0
3313,0


In [ ]:
X_train=pd.DataFrame(X_train)
X_train

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
1069,22,510,0,1,0,160.4,108,27.27,218.1,88,18.54,192.9,115,8.68,12.5,4,3.38,1
1582,86,510,0,1,0,150.5,92,25.59,120.3,95,10.23,271.2,96,12.20,9.0,2,2.43,1
2444,121,415,0,0,31,263.1,70,44.73,279.3,118,23.74,127.1,143,5.72,9.7,4,2.62,5
3196,124,510,0,1,0,169.3,108,28.78,178.6,91,15.18,242.3,82,10.90,12.2,3,3.29,1
267,119,408,0,1,0,138.3,89,23.51,170.5,78,14.49,263.9,98,11.88,13.5,6,3.65,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,21,415,0,1,0,146.0,78,24.82,109.7,79,9.32,247.4,108,11.13,6.8,7,1.84,0
1414,182,415,0,1,0,176.1,90,29.94,174.9,106,14.87,234.7,134,10.56,9.7,4,2.62,1
2191,113,415,0,1,0,90.6,130,15.40,170.6,100,14.50,137.4,74,6.18,5.4,9,1.46,1
3313,127,408,0,1,0,102.8,128,17.48,143.7,95,12.21,191.4,97,8.61,10.0,5,2.70,1


In [ ]:
y_train.shape

(2333, 1)

In [ ]:
result = pd.concat([X_train, y_train], axis=1)
result

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
1069,22,510,0,1,0,160.4,108,27.27,218.1,88,18.54,192.9,115,8.68,12.5,4,3.38,1,0
1582,86,510,0,1,0,150.5,92,25.59,120.3,95,10.23,271.2,96,12.20,9.0,2,2.43,1,0
2444,121,415,0,0,31,263.1,70,44.73,279.3,118,23.74,127.1,143,5.72,9.7,4,2.62,5,0
3196,124,510,0,1,0,169.3,108,28.78,178.6,91,15.18,242.3,82,10.90,12.2,3,3.29,1,0
267,119,408,0,1,0,138.3,89,23.51,170.5,78,14.49,263.9,98,11.88,13.5,6,3.65,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,21,415,0,1,0,146.0,78,24.82,109.7,79,9.32,247.4,108,11.13,6.8,7,1.84,0,0
1414,182,415,0,1,0,176.1,90,29.94,174.9,106,14.87,234.7,134,10.56,9.7,4,2.62,1,0
2191,113,415,0,1,0,90.6,130,15.40,170.6,100,14.50,137.4,74,6.18,5.4,9,1.46,1,0
3313,127,408,0,1,0,102.8,128,17.48,143.7,95,12.21,191.4,97,8.61,10.0,5,2.70,1,0


In [ ]:
from pycaret.classification import *
s = setup(data = result, target = 'Churn', session_id=123)

,Description,Value
0,session_id,123
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(2333, 19)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9351,0.9007,0.6922,0.8778,0.7693,0.7324,0.7422,0.364
lightgbm,Light Gradient Boosting Machine,0.9308,0.8828,0.6414,0.8992,0.7411,0.7030,0.7206,0.143
rf,Random Forest Classifier,0.9143,0.9030,0.5049,0.9179,0.6434,0.6004,0.6389,0.695
dt,Decision Tree Classifier,0.9039,0.8160,0.6878,0.6974,0.6907,0.6339,0.6352,0.046
et,Extra Trees Classifier,0.8879,0.8888,0.3798,0.8247,0.5038,0.4527,0.5033,0.520
ada,Ada Boost Classifier,0.8733,0.8623,0.4268,0.6573,0.5128,0.4440,0.4602,0.172
lr,Logistic Regression,0.8512,0.8696,0.2895,0.5526,0.3750,0.3010,0.3231,1.194
lda,Linear Discriminant Analysis,0.8512,0.8750,0.4142,0.5327,0.4634,0.3792,0.3845,0.026
ridge,Ridge Classifier,0.8445,0.0000,0.2735,0.5249,0.3533,0.2752,0.2970,0.028
dummy,Dummy Classifier,0.8432,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.015
